In [1]:
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from tensorflow import keras

In [2]:
def model(hp):
    denseLayer = hp.Int("densel", min_value=3, max_value=1023, step=3)
    kernelSize1 = hp.Int("kernelSize1", min_value=2, max_value=256, step=2)
    kernelSize2 = hp.Int("kernelSize2", min_value=2, max_value=256, step=2)
    kernelSize3 = hp.Int("kernelSize3", min_value=2, max_value=256, step=2)
    kernelSize4 = hp.Int("kernelSize4", min_value=2, max_value=256, step=2)
    
    
    METRICS = [
      #keras.metrics.TruePositives(name='tp', thresholds=threshold),
      #keras.metrics.FalsePositives(name='fp', thresholds=threshold),
      #keras.metrics.TrueNegatives(name='tn',thresholds=threshold),
      #keras.metrics.FalseNegatives(name='fn',thresholds=threshold), 
      keras.metrics.BinaryAccuracy(name='accuracy',threshold=threshold),
      keras.metrics.Precision(name='precision',thresholds=threshold),
      keras.metrics.Recall(name='recall',thresholds=threshold),
      #tfa.metrics.FBetaScore(name="fbeta", num_classes=1,beta=float(2),average='macro')
      #keras.metrics.SpecificityAtSensitivity(sensitivity=.95, name='SensAt95',threshold=threshold)
      #keras.metrics.AUC(name='auc'),
      #keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(kernelSize1, (3,3), activation='relu', input_shape=(128, 128, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(kernelSize2, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(kernelSize3, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(kernelSize4, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(int(denseLayer/4/2*3), (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(denseLayer, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])
                         
                         
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=METRICS,
    )
                         
    return(model)                     

In [3]:
image_size = (128, 128)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-training-and-validation",
    validation_split=0.25,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode = "grayscale"
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-training-and-validation",
    validation_split=0.25,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode="grayscale"
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-testing",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode="grayscale"
)

Found 17600 files belonging to 2 classes.
Using 13200 files for training.
Found 17600 files belonging to 2 classes.
Using 4400 files for validation.
Found 4401 files belonging to 2 classes.


In [4]:
tuner = kt.Hyperband(
    model,
    objective='val_loss',
    max_epochs=35)

In [5]:
tuner.search(train_ds, epochs=5, validation_data=val_ds)

Trial 90 Complete [00h 15m 42s]
val_loss: 0.047968100756406784

Best val_loss So Far: 0.03656580671668053
Total elapsed time: 05h 35m 35s
INFO:tensorflow:Oracle triggered exit


In [6]:
best_hyperparameters = tuner.get_best_hyperparameters()

In [3]:
for hyperparameters in best_hyperparameters:
    print(hyperparameters.get("kernelSize1"))
    print(hyperparameters.get("kernelSize2"))
    print(hyperparameters.get("kernelSize3"))
    print(hyperparameters.get("kernelSize4"))
    print(hyperparameters.get("densel"))

222
196
74
168
89
237
